### 1. Import packages

In [1]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")

import gym

### 2. Helper functions

In [2]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    
    return state


In [3]:
def dict2array_partial(state):
    new_state = []
    num_observable_states = 0
    for key  in state.keys():
        if key != 'sw':
            if key == 'cumsumfert':
                new_state.append(state[key])
            if key == 'dap':
                new_state.append(state[key])
            if key == 'dtt':
                new_state.append(state[key])
            if key == 'istage':
                new_state.append(state[key])
            if key == 'pltpop':
                new_state.append(state[key])
            if key == 'rain':
                new_state.append(state[key])
            if key == 'srad':
                new_state.append(state[key])
            if key == 'tmax':
                new_state.append(state[key])
            if key == 'tmin':
                new_state.append(state[key])
            if key == 'vstage':
                new_state.append(state[key])
            if key == 'xlai':
                new_state.append(state[key])
        else:
            new_state += list(state['sw'])
    state = np.asarray(new_state)
    return state

### 3. Initialize the environment

In [4]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
    #'fileX_template_path':'./IUAF9901.MZX',
    #'experiment_number':1
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = 20
nb_actions = 2
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 9

Ram information received from DASSAT will has 20 dimensions.
There are 2 possible actions at each step.
Discrete? False


### 4. Define the network

In [5]:
class Partial_QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=128*2,fc2_units=128*2,fc3_units=128*2):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(Partial_QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc4 = nn.Linear(fc3_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        y = F.relu(self.fc2(x))
        z = F.relu(self.fc3(y))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return 4*torch.sigmoid(self.fc4(z))*torch.tensor([[float(40),float(6)]])

### 5. Define the Replay Buffer

In [6]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [7]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 640         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [8]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, trained_model):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size
        # Q-Network
        self.qnetwork_local = trained_model
        #self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())

### DQN for nitrogen management

In [9]:
def get_W(water):
    return round(water/6)*6

In [10]:
def get_N(nitrogen):
    return round(nitrogen/40)*40

In [15]:
trained_model = Partial_QNetwork(20, 2)
trained_model.load_state_dict(torch.load('/home/rant3/focal/IAAI/Trained_Policies/FL_Super_Partial_Both/par_10.pth'))

<All keys matched successfully>

In [16]:
for i in range(1):
    Action_list=[]
    water_list=[]
    Nleach=0
    Yield=0
    reward=0
    #print('')
    env.reset()
    n_amount=0
    w_amount=0
    #print(env.observation)
    #date = 1
    #print(date)
    while not env.done:
        #print(observation)
        #print(observation)
        # observation_list = env.observation_dict_to_array(observation)
        state=env.observation
        full_state = dict2array(state)
        partial_state = dict2array_partial(state)
        partial_state = torch.from_numpy(partial_state).float().unsqueeze(0).to(device)
        action1=trained_model(partial_state)
        #print(action1[0][0].item())
        N1=(action1[0][0].item())
        W1=(action1[0][1].item())
        N2=get_N(action1[0][0].item())
        W2=get_W(action1[0][1].item())
        #print(action1)
        #print(action1)
        action = {
                    'anfer': N2,# if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': W2, # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
        n_amount += N2
        w_amount += W2
        Action_list.append(N2)
        water_list.append(W2)

        observation, reward, done, info = env.step(action)
        #print('date is ',env.observation['dap'])
        #print('N1 is ',round(N1))
        #print('N2 IS ', N2)
        #print('observation is', observation)
        #real_ob = add_fert(observation, action_dict['anfer'])
        #print('real_ob is ', real_ob)
        #env.observation = real_ob
        #print('final ob is:', env.observation)
        #real_s=get_real_state(env._state, action_dict['anfer'])
        #env._state=real_s
        #print('env_state', env._state)
        #print(date)
        #print(reward)
        #print(done)
        if done:
            final_ob=env.observation
            n=final_ob['cumsumfert']
            #print(final_ob)
            Nleach=final_ob['cleach']
            Yield=final_ob['grnwt']
            reward=Yield*0.2-1*n_amount-1*w_amount-5*Nleach
            #print(env.observation)
            #print(env.history['action'])
            #print(env.history['reward'])
print(final_ob)
print('N amount is', n_amount)
print(n)
#print(Action_list)
#print('avergae N input is', np.mean(Action_list))
print('')
print('water amount is', w_amount)
#print(water_list)
#print('avergae water amount is', np.mean(water_list))
print('')
print('N leaching is', Nleach)
#print('avergae Leaching is', np.mean(Nleach_list))
print('')
print('Yield is', Yield)
#print('avergae Yield is', np.mean(Yield_list))
print('')
print('Reward is', reward)
#print('avergae reward is', np.mean(reward_list))


{'cleach': 39.23649978637695, 'cnox': 0.5346603989601135, 'cumsumfert': 200.0, 'dap': 126, 'dtt': 20.900001525878906, 'es': 0.32613709568977356, 'grnwt': 10926.050247667881, 'istage': 6, 'nstres': 0.5112686157226562, 'pcngrn': 0.014136134386062623, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 103.29289245605469, 'runoff': 0.0, 'srad': 18.700000762939453, 'sw': array([0.06494 , 0.061098, 0.061772, 0.065845, 0.064315, 0.085602,
       0.0904  , 0.130675, 0.25957 ]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 33.900001525878906, 'tmin': 23.899999618530273, 'tnoxd': 0.0021152952685952187, 'topwt': 2045.35986328125, 'trnu': 0.6905401911195064, 'vstage': 20.0, 'wtdep': 179.53822326660156, 'wtnup': 207.37586975097656, 'xlai': 1.6181052923202515}
N amount is 200
200.0

water amount is 138

N leaching is 39.23649978637695

Yield is 10926.050247667881

Reward is 1651.0275506016915


In [13]:
import pandas as pd
a = pd.DataFrame(Action_list)
a.to_csv('Action.csv', index=False, header=False)
b = pd.DataFrame(water_list)
b.to_csv('water.csv', index=False, header=False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(topwt)
plt.title("topwt paper method k=0.1 act 40")
plt.savefig("topwt paper method k=0.1 act 40.png")
plt.figure()
plt.plot(nitrogen_uptake)
plt.title("Nitrogen uptake paper method k=0.1 act 40")
plt.savefig("Nitrogen uptake paper method k=0.1 act 40.png")
plt.figure()
plt.plot(leaching)
plt.title("Nitrate leaching paper method k=0.1 act 40")
plt.savefig("Nitrate leaching paper method k=0.1 act 40.png")
plt.figure()
plt.plot(action_list)
plt.title("action paper method k=0.1 act 40")
plt.savefig("action paper method k=0.1 act 40.png")

In [14]:
import pandas as pd
a = pd.DataFrame(topwt)
a.to_csv('topwt paper method k=0.1 act 40.csv', index=False, header=False)
b = pd.DataFrame(nitrogen_uptake)
b.to_csv('Nitrogen uptake paper method k=0.1 act 40.csv', index=False, header=False)
c = pd.DataFrame(leaching)
c.to_csv('Nitrate leaching paper method k=0.1 act 40.csv', index=False, header=False)
c = pd.DataFrame(action_list)
c.to_csv('Action paper method k=0.1 act 40.csv', index=False, header=False)

In [15]:
score

2097.3241743206454

In [12]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(action_list)
plt.title("action k=0.1 act every 14 days")
plt.savefig("action k=0.1 act every 14 days.png")